# 실습 [9-1]<br>
**실습명: N-gram 언어 모델로 문장 생성**<br>
: 특정 단어가 등장하는 확률을 계산할 시에 이전 N-1개의 단어가 등장할 확률을 고려

1. NLTK 패키지 사용해 입력 텍스트를 N-gram으로 바꾸기 (데이터: nsmc 영화 리뷰)
2. 단어의 빈도 측정
3. 조건부 확률 추정
4. 단어열 예측하고 생성하는 모델 구축

In [ ]:
#한국어 처리에 필요한 konlpy 패키지 설치 전 선행파일 설치
!apt-get update
!apt-get install g++ openjdk-8 -jdk python-dev python3-dev
!pip3 install JPype1-py3
!pip3 install konlpy
!JAVA_HOME = "/usr/lib/jvm/java-8-openjdk-amd64"
from konlpy.tag import Okt

In [2]:
import nltk
import konlpy

from nltk.util import ngrams
from nltk import word_tokenize
from nltk import ConditionalFreqDist #단어별 등장 빈도
from nltk.probability import ConditionalProbDist, MLEProbDist #조건부확률, MLE확률

import codecs
from konlpy.tag import Okt #KoNLPy 패키지의 okt 토큰화기/형태소 분석기
from tqdm import tqdm

In [3]:
#1. 입력 텍스트 N-gram 형태로 바꾸기
sentence = "나는 매일 아침 지하철을 탄다"

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
#입력 테스트 띄어쓰기 단위 토큰화
tokens = word_tokenize(sentence)
print(tokens)

['나는', '매일', '아침', '지하철을', '탄다']


In [5]:
#하지만 한국어는 교착어, 띄어쓰기 단위로 토큰화하지 않음
#konlpy의 Okt를 이용해서 형태소 기준 토큰화
tagger = Okt()

def tokenize(text):
  tokens = ['/'.join(t) for t in tagger.pos(text)]
  return tokens

tokens = tokenize(sentence)
print(tokens)

['나/Noun', '는/Josa', '매일/Noun', '아침/Noun', '지하철/Noun', '을/Josa', '탄다/Verb']


In [6]:
#토큰 N-gram 형태로 바꿔주기 (ngrams 2번째 인자에 N 지정)
bigram = ngrams(tokens, 2)
trigram = ngrams(tokens, 3)

In [7]:
print("bigram")
for b in bigram:
  print(b)

print("trigram")
for t in trigram:
  print(t)

bigram
('나/Noun', '는/Josa')
('는/Josa', '매일/Noun')
('매일/Noun', '아침/Noun')
('아침/Noun', '지하철/Noun')
('지하철/Noun', '을/Josa')
('을/Josa', '탄다/Verb')
trigram
('나/Noun', '는/Josa', '매일/Noun')
('는/Josa', '매일/Noun', '아침/Noun')
('매일/Noun', '아침/Noun', '지하철/Noun')
('아침/Noun', '지하철/Noun', '을/Josa')
('지하철/Noun', '을/Josa', '탄다/Verb')


In [8]:
#padding을 통해 입력 데이터 문장 시작,끝을 알리는 토큰 추가 (<s>, </s>)
bigram = ngrams(tokens, 2, pad_left=True, pad_right=True, left_pad_symbol="<s>", right_pad_symbol="</s>")
print("bigrams with padding : ")
for b in bigram:
  print(b)

bigrams with padding : 
('<s>', '나/Noun')
('나/Noun', '는/Josa')
('는/Josa', '매일/Noun')
('매일/Noun', '아침/Noun')
('아침/Noun', '지하철/Noun')
('지하철/Noun', '을/Josa')
('을/Josa', '탄다/Verb')
('탄다/Verb', '</s>')


In [9]:
#네이버 영화 리뷰 데이터셋 다운로드
%%time
!wget -nc -q https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt

CPU times: user 14.7 ms, sys: 11.2 ms, total: 25.9 ms
Wall time: 1.13 s


In [10]:
#다운로드 받은 데이터셋 읽기 (인덱스, 라벨 제외한 텍스트 부분만 가져오기)
#codecs: 대용량 파일 조금씩 읽어주는 라이브러리

with codecs.open("ratings_train.txt", encoding='utf-8') as f:
  data = [line.split('\t') for line in f.read().splitlines()] #\n 제외
  data = data[1:] #header 제외
print("데이터셋 : ", data[:10])
docs = [row[1] for row in data] #텍스트 부분만 추출
print("텍스트 데이터 : ", docs[:5])
print("문장 개수 : ", len(docs))

데이터셋 :  [['9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0'], ['3819312', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '1'], ['10265843', '너무재밓었다그래서보는것을추천한다', '0'], ['9045019', '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '0'], ['6483659', '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다', '1'], ['5403919', '막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.', '0'], ['7797314', '원작의 긴장감을 제대로 살려내지못했다.', '0'], ['9443947', '별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네', '0'], ['7156791', '액션이 없는데도 재미 있는 몇안되는 영화', '1'], ['5912145', '왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?', '1']]
텍스트 데이터 :  ['아 더빙.. 진짜 짜증나네요 목소리', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '너무재밓었다그래서보는것을추천한다', '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다']
문장 개수 :  150000


In [11]:
#토큰화한 데이터의 bigram 리스트에 추가
sentences = []

for d in tqdm(docs):
  tokens = tokenize(d)
  bigram = ngrams(tokens, 2, pad_left=True, pad_right=True, left_pad_symbol="<s>", right_pad_symbol="</s>")
  sentences += [t for t in bigram]

100%|██████████| 150000/150000 [06:42<00:00, 372.42it/s]


In [12]:
print(sentences[:10]) #bigram으로 토큰화한 단어 10개만 불러오기

[('<s>', '아/Exclamation'), ('아/Exclamation', '더빙/Noun'), ('더빙/Noun', '../Punctuation'), ('../Punctuation', '진짜/Noun'), ('진짜/Noun', '짜증나네요/Adjective'), ('짜증나네요/Adjective', '목소리/Noun'), ('목소리/Noun', '</s>'), ('<s>', '흠/Noun'), ('흠/Noun', '.../Punctuation'), ('.../Punctuation', '포스터/Noun')]


In [13]:
#2. sentence 단어별 빈도 측정
cfd = ConditionalFreqDist(sentences)

In [14]:
#<s> 다음에 가장 많이 오는 단어 5개 출력
print(cfd["<s>"].most_common(5))

[('정말/Noun', 2718), ('이/Noun', 2371), ('진짜/Noun', 2232), ('이/Determiner', 2115), ('영화/Noun', 2069)]


In [15]:
#특정 단어 c 다음에 가장 많이 등장하는 n개 단어 반환
def most_common(c, n, pos=None):
  if pos is None:
    return cfd[tokenize(c)[0]].most_common(n)
  else:
    return cfd["/".join([c, pos]).most_common(n)]

print(most_common("나", 10)) #"나" 다음에 가장 많이 오는 단어 10개 출력

In [16]:
#3. 단어별 등장빈도 기반 조건부 확률 추정
cpd = ConditionalProbDist(cfd, MLEProbDist)

In [17]:
#"." 다음에 "</s>" 가 올 확률 출력
print(cpd[tokenize(".")[0]].prob("</s>"))

0.39102658679807606


In [20]:
#토큰 c 다음에 토큰 w가 bigram에 등장할 확률
def bigram_prob(c,w):
  context = tokenize(c)[0]
  word = tokenize(w)[0]
  return cpd[context].prob(word)

print(bigram_prob("영화", "이"))

0.00015767585785521414


In [23]:
print(bigram_prob("이", "영화"))

0.4010748656417948


In [25]:
#4. 조건부 확률이 가장 높은 토큰열을 토대로 문장 생성하는 알고리즘
def generate_sentences(seed=None, debug=False):
  if seed is not None:
    import random
    random.seed(seed) #무작위 seed 정하기
  c = "<s>"
  sentence = []
  while True:
    if c not in cpd: #다음에 등장할 확률이 0이면 break
      break
    w = cpd[c].generate() #w: 가장 확률이 높은 토큰열 w
    if w == "</s>": #문장이 끝나면 break
      break
    
    word = w.split("/")[0]
    pos = w.split("/")[1]
    
    #조사,어미를 제외하고 각 토큰은 띄어쓰기로 구분해서 생성
    if c == "<s>":
      sentence.append(word.title())
    elif c in ["`", "\"","'","("]: #특정 punctuation이 등장하면 넘어가기
      sentence.append(word)
    elif word in ["'", ".", ",", ")", ":", ";", "?"]: 
      sentence.append(word)
    elif pos in ["Josa", "Punctuation", "Suffix"]: #조사, 기호, 어미 나오면 넘어가기
        sentence.append(word)
    elif w in ["임/Noun", "것/Noun", "는걸/Noun", "릴때/Noun",
                "되다/Verb", "이다/Verb", "하다/Verb", "이다/Adjective"]: #넘어가지 않을 조사/어미 정의
        sentence.append(word)
    else:
        sentence.append(" " + word)
    c = w
    if debug:
      print(w)
  return "".join(sentence)

print(generate_sentences(2))

도리까지 본 영화 너무... 뭔가.. 최고네요. 하지만.. 눈물 낫다는건 또 영화에 들지 않는다. 근데 뭐야 어떻게 그렇게 착했던 윤재랑은 에바 그린 드레스 소리 듣는거임""" 에리 욧의 미모로 합성 한 가수 노래와 흥행 놓친 영화다. 사투리 연기 하나 없는 ‘ 스피드 감 넘치는 스릴 넘치는 연기를 이해 되지 못 하시는 분보다 훨 재밌구만 평점을 망처 놓은 듯하다. 영화 보는이로 하여금 불편함을 느꼇을듯


In [26]:
#debug=True하면 각각 토큰화된 form도 함께 보여줌
print(generate_sentences(2, debug=True))

도리/Noun
까지/Josa
본/Verb
영화/Noun
너무/Adverb
.../Punctuation
뭔가/Noun
../Punctuation
최고/Noun
네/Suffix
요/Josa
./Punctuation
하지만/Conjunction
../Punctuation
눈물/Noun
낫다는건/Verb
또/Noun
영화/Noun
에/Josa
들지/Verb
않는다/Verb
./Punctuation
근데/Adverb
뭐/Noun
야/Josa
어떻게/Adjective
그렇게/Adverb
착했던/Adjective
윤재/Noun
랑은/Josa
에바/Noun
그린/Noun
드레스/Noun
소리/Noun
듣는거/Verb
임/Noun
"""/Punctuation
에리/Noun
욧의/Noun
미모/Noun
로/Josa
합성/Noun
한/Determiner
가수/Noun
노래/Noun
와/Josa
흥행/Noun
놓친/Verb
영화/Noun
다/Josa
./Punctuation
사투리/Noun
연기/Noun
하나/Noun
없는/Adjective
‘/Foreign
스피드/Noun
감/Noun
넘치는/Adjective
스릴/Noun
넘치는/Adjective
연기/Noun
를/Josa
이해/Noun
되지/Verb
못/VerbPrefix
하시는/Verb
분/Noun
보다/Josa
훨/Noun
재밌구만/Adjective
평점/Noun
을/Josa
망처/Noun
놓은/Verb
듯/Noun
하다/Verb
./Punctuation
영화/Noun
보는이로/Verb
하여금/Adverb
불편함을/Adjective
느꼇을듯/Noun
도리까지 본 영화 너무... 뭔가.. 최고네요. 하지만.. 눈물 낫다는건 또 영화에 들지 않는다. 근데 뭐야 어떻게 그렇게 착했던 윤재랑은 에바 그린 드레스 소리 듣는거임""" 에리 욧의 미모로 합성 한 가수 노래와 흥행 놓친 영화다. 사투리 연기 하나 없는 ‘ 스피드 감 넘치는 스릴 넘치는 연기를 이해 되지 못 하시는 분보다 훨 재밌구만 평점을 망처 놓은 듯하다. 영화 보는이